In [46]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD

import warnings
warnings.filterwarnings("ignore")

# Import data and preprocess
data = pd.read_csv('customer_data.csv')
data = data.dropna()
data = data.drop_duplicates()

# Define the model structure
model = BayesianModel([('Age', 'Purchase'), ('Gender', 'Purchase'), ('History', 'Purchase')])

# Define the CPDs
cpd_age = MaximumLikelihoodEstimator(model, data).estimate_cpd('Age')
cpd_gender = MaximumLikelihoodEstimator(model, data).estimate_cpd('Gender')
cpd_history = MaximumLikelihoodEstimator(model, data).estimate_cpd('History')
cpd_purchase = BayesianEstimator(model, data).estimate_cpd('Purchase', prior_type='BDeu', equivalent_sample_size=10)

# Add the CPDs to the model
model.add_cpds(cpd_age, cpd_gender, cpd_history, cpd_purchase)

# Check if the model is correct
model.check_model()

# Instantiate the variable elimination method for inference
model_checker = VariableElimination(model)

# Perform inference
result = model_checker.query(variables=['Purchase'], evidence={'Age': 32, 'Gender': 'Male', 'History': 'High'})
print(result)


+----------------+-----------------+
| Purchase       |   phi(Purchase) |
+================+=================+
| Purchase(500)  |          0.0214 |
+----------------+-----------------+
| Purchase(800)  |          0.0214 |
+----------------+-----------------+
| Purchase(900)  |          0.0214 |
+----------------+-----------------+
| Purchase(1000) |          0.0214 |
+----------------+-----------------+
| Purchase(1200) |          0.8291 |
+----------------+-----------------+
| Purchase(1300) |          0.0214 |
+----------------+-----------------+
| Purchase(1500) |          0.0214 |
+----------------+-----------------+
| Purchase(3940) |          0.0214 |
+----------------+-----------------+
| Purchase(4440) |          0.0214 |
+----------------+-----------------+
